# ExtraSensory Transfer Entropy Analysis Report

## Research Question
This analysis investigates the asymmetric predictive relationship between physical activity level (A) and sitting state (S) using the ExtraSensory dataset.

**Hypothesis H₁**: Activity level (A) is more predictive of an upcoming transition into a sitting state (S) than the reverse.

**Formal Statement**: E[ΔTE] > 0, where ΔTE = TE(A → S) - TE(S → A)

## Methodology
- **Variable A (Activity)**: Z-scored and quantile-discretized accelerometer magnitude {0,1,2,3,4}
- **Variable S (Sitting)**: Binary sitting state {0,1}
- **Variable H (Hour)**: Hour of day {0-23} for robustness check
- **Analysis**: Transfer Entropy (TE) and Conditional Transfer Entropy (CTE) using JIDT
- **Optimization**: History length k optimized via Active Information Storage (AIS)
- **Significance**: Permutation testing with 1000 surrogates


In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import wilcoxon, describe
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11


## 1. Load Results Data

Load the aggregated results from the batch processing pipeline.


In [ ]:
# Load results CSV
results_file = 'results/extrasensory_te_results.csv'
df = pd.read_csv(results_file)

print(f"Total subjects processed: {len(df)}")
print(f"\nDataFrame shape: {df.shape}")
print(f"\nColumns: {list(df.columns)}")


In [ ]:
# Display first few rows
df.head(10)


## 2. Descriptive Statistics

Summary statistics for key variables across all subjects.


In [ ]:
# Data length statistics
print("=" * 60)
print("DATA LENGTH STATISTICS")
print("=" * 60)
print(df['data_length'].describe())


In [ ]:
# History length optimization results
print("=" * 60)
print("OPTIMIZED HISTORY LENGTHS (k)")
print("=" * 60)
print("\nk_A (Activity):")
print(df['k_A'].describe())
print(f"\nMode: {df['k_A'].mode().values}")

print("\nk_S (Sitting):")
print(df['k_S'].describe())
print(f"\nMode: {df['k_S'].mode().values}")


In [ ]:
# Transfer Entropy statistics
print("=" * 60)
print("TRANSFER ENTROPY (TE) STATISTICS")
print("=" * 60)
print("\nTE(A→S):")
print(df['TE(A->S)'].describe())

print("\nTE(S→A):")
print(df['TE(S->A)'].describe())

print("\nΔTE = TE(A→S) - TE(S→A):")
print(df['Delta_TE'].describe())

# Count significant results (p < 0.05)
sig_A_to_S = (df['p(A->S)'] < 0.05).sum()
sig_S_to_A = (df['p(S->A)'] < 0.05).sum()
print(f"\nSignificant TE(A→S) results (p<0.05): {sig_A_to_S}/{len(df)} ({100*sig_A_to_S/len(df):.1f}%)")
print(f"Significant TE(S→A) results (p<0.05): {sig_S_to_A}/{len(df)} ({100*sig_S_to_A/len(df):.1f}%)")


In [ ]:
# Conditional Transfer Entropy statistics
print("=" * 60)
print("CONDITIONAL TRANSFER ENTROPY (CTE) STATISTICS")
print("=" * 60)
print("\nCTE(A→S|H):")
print(df['CTE(A->S|H)'].describe())

print("\nCTE(S→A|H):")
print(df['CTE(S->A|H)'].describe())

print("\nΔCTE = CTE(A→S|H) - CTE(S→A|H):")
print(df['Delta_CTE'].describe())

# Count significant results (p < 0.05)
sig_cte_A_to_S = (df['p_cte(A->S|H)'] < 0.05).sum()
sig_cte_S_to_A = (df['p_cte(S->A|H)'] < 0.05).sum()
print(f"\nSignificant CTE(A→S|H) results (p<0.05): {sig_cte_A_to_S}/{len(df)} ({100*sig_cte_A_to_S/len(df):.1f}%)")
print(f"Significant CTE(S→A|H) results (p<0.05): {sig_cte_S_to_A}/{len(df)} ({100*sig_cte_S_to_A/len(df):.1f}%)")


## 3. Statistical Hypothesis Testing

### 3.1 Primary Hypothesis: ΔTE > 0

We test whether the mean difference in transfer entropy (ΔTE) is significantly greater than zero using a one-sample Wilcoxon signed-rank test.

**Null Hypothesis (H₀)**: E[ΔTE] = 0 (no asymmetry in predictive relationship)

**Alternative Hypothesis (H₁)**: E[ΔTE] > 0 (activity is more predictive of sitting than vice versa)


In [ ]:
# Remove NaN values for statistical testing
delta_te_clean = df['Delta_TE'].dropna()

print(f"Sample size for ΔTE test: N = {len(delta_te_clean)}")
print(f"Missing values: {df['Delta_TE'].isna().sum()}")

# Wilcoxon signed-rank test (one-tailed: alternative='greater')
statistic, p_value = wilcoxon(delta_te_clean, alternative='greater')

print("\n" + "=" * 60)
print("WILCOXON SIGNED-RANK TEST FOR ΔTE")
print("=" * 60)
print(f"Test statistic: {statistic:.2f}")
print(f"p-value (one-tailed): {p_value:.6f}")
print(f"Significance level: α = 0.05")

if p_value < 0.05:
    print(f"\n✓ RESULT: REJECT H₀ (p = {p_value:.6f} < 0.05)")
    print("Conclusion: Significant evidence that E[ΔTE] > 0")
    print("Activity level is significantly more predictive of sitting than vice versa.")
else:
    print(f"\n✗ RESULT: FAIL TO REJECT H₀ (p = {p_value:.6f} ≥ 0.05)")
    print("Conclusion: Insufficient evidence that E[ΔTE] > 0")
    print("Cannot conclude that activity is more predictive of sitting.")

# Effect size: median and mean
print(f"\nMedian ΔTE: {delta_te_clean.median():.6f}")
print(f"Mean ΔTE: {delta_te_clean.mean():.6f}")
print(f"Std ΔTE: {delta_te_clean.std():.6f}")


### 3.2 Robustness Check: ΔCTE > 0

We verify the result holds when conditioning on hour-of-day (H) to control for circadian patterns.

**Null Hypothesis (H₀)**: E[ΔCTE] = 0

**Alternative Hypothesis (H₁)**: E[ΔCTE] > 0


In [ ]:
# Remove NaN values for CTE testing
delta_cte_clean = df['Delta_CTE'].dropna()

print(f"Sample size for ΔCTE test: N = {len(delta_cte_clean)}")
print(f"Missing values: {df['Delta_CTE'].isna().sum()}")

# Wilcoxon signed-rank test (one-tailed)
statistic_cte, p_value_cte = wilcoxon(delta_cte_clean, alternative='greater')

print("\n" + "=" * 60)
print("WILCOXON SIGNED-RANK TEST FOR ΔCTE (ROBUSTNESS)")
print("=" * 60)
print(f"Test statistic: {statistic_cte:.2f}")
print(f"p-value (one-tailed): {p_value_cte:.6f}")
print(f"Significance level: α = 0.05")

if p_value_cte < 0.05:
    print(f"\n✓ RESULT: REJECT H₀ (p = {p_value_cte:.6f} < 0.05)")
    print("Conclusion: Asymmetry persists when controlling for hour-of-day")
    print("The result is robust to circadian confounding.")
else:
    print(f"\n✗ RESULT: FAIL TO REJECT H₀ (p = {p_value_cte:.6f} ≥ 0.05)")
    print("Conclusion: Asymmetry does not persist when controlling for hour-of-day")
    print("The result may be confounded by circadian patterns.")

# Effect size
print(f"\nMedian ΔCTE: {delta_cte_clean.median():.6f}")
print(f"Mean ΔCTE: {delta_cte_clean.mean():.6f}")
print(f"Std ΔCTE: {delta_cte_clean.std():.6f}")


## 4. Visualizations

### 4.1 Distribution of ΔTE


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram with KDE
axes[0].hist(delta_te_clean, bins=30, alpha=0.6, color='steelblue', edgecolor='black', density=True)
delta_te_clean.plot(kind='kde', ax=axes[0], color='darkblue', linewidth=2)
axes[0].axvline(0, color='red', linestyle='--', linewidth=2, label='H₀: ΔTE = 0')
axes[0].axvline(delta_te_clean.median(), color='green', linestyle='--', linewidth=2, label=f'Median = {delta_te_clean.median():.4f}')
axes[0].set_xlabel('ΔTE = TE(A→S) - TE(S→A)', fontsize=12)
axes[0].set_ylabel('Density', fontsize=12)
axes[0].set_title('Distribution of Net Transfer Entropy (ΔTE)', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Violin plot
axes[1].violinplot([delta_te_clean], positions=[1], showmeans=True, showmedians=True)
axes[1].axhline(0, color='red', linestyle='--', linewidth=2, label='H₀: ΔTE = 0')
axes[1].set_ylabel('ΔTE = TE(A→S) - TE(S→A)', fontsize=12)
axes[1].set_title('Violin Plot of ΔTE', fontsize=14, fontweight='bold')
axes[1].set_xticks([1])
axes[1].set_xticklabels(['All Subjects'])
axes[1].legend()
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()


### 4.2 Paired Comparison: TE(A→S) vs TE(S→A)


In [ ]:
# Scatter plot with identity line
fig, ax = plt.subplots(figsize=(10, 10))

# Remove NaN values for plotting
plot_data = df[['TE(A->S)', 'TE(S->A)']].dropna()

ax.scatter(plot_data['TE(S->A)'], plot_data['TE(A->S)'], alpha=0.6, s=50, color='steelblue', edgecolors='black')

# Identity line (TE(A→S) = TE(S→A))
max_val = max(plot_data['TE(A->S)'].max(), plot_data['TE(S->A)'].max())
min_val = min(plot_data['TE(A->S)'].min(), plot_data['TE(S->A)'].min())
ax.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Identity Line (TE(A→S) = TE(S→A))')

ax.set_xlabel('TE(S→A): Sitting predicts Activity', fontsize=12)
ax.set_ylabel('TE(A→S): Activity predicts Sitting', fontsize=12)
ax.set_title('Paired Comparison of Transfer Entropy', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

# Annotate number of points above/below identity line
above_line = (plot_data['TE(A->S)'] > plot_data['TE(S->A)']).sum()
below_line = (plot_data['TE(A->S)'] < plot_data['TE(S->A)']).sum()
ax.text(0.05, 0.95, f'Above line (A→S > S→A): {above_line}\nBelow line (A→S < S→A): {below_line}',
        transform=ax.transAxes, fontsize=11, verticalalignment='top',
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()

print(f"Points above identity line (TE(A→S) > TE(S→A)): {above_line}/{len(plot_data)} ({100*above_line/len(plot_data):.1f}%)")
print(f"Points below identity line (TE(A→S) < TE(S→A)): {below_line}/{len(plot_data)} ({100*below_line/len(plot_data):.1f}%)")


### 4.3 Robustness: TE vs CTE Comparison


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# ΔTE vs ΔCTE comparison
delta_comparison = df[['Delta_TE', 'Delta_CTE']].dropna()

axes[0].scatter(delta_comparison['Delta_TE'], delta_comparison['Delta_CTE'], 
                alpha=0.6, s=50, color='steelblue', edgecolors='black')
axes[0].axhline(0, color='red', linestyle='--', linewidth=1, alpha=0.5)
axes[0].axvline(0, color='red', linestyle='--', linewidth=1, alpha=0.5)

# Diagonal line
max_val = max(delta_comparison['Delta_TE'].max(), delta_comparison['Delta_CTE'].max())
min_val = min(delta_comparison['Delta_TE'].min(), delta_comparison['Delta_CTE'].min())
axes[0].plot([min_val, max_val], [min_val, max_val], 'g--', linewidth=2, alpha=0.5, label='ΔTE = ΔCTE')

axes[0].set_xlabel('ΔTE (unconditional)', fontsize=12)
axes[0].set_ylabel('ΔCTE (conditional on hour)', fontsize=12)
axes[0].set_title('ΔTE vs ΔCTE: Robustness Check', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Side-by-side violin plots
axes[1].violinplot([delta_te_clean, delta_cte_clean], positions=[1, 2], showmeans=True, showmedians=True)
axes[1].axhline(0, color='red', linestyle='--', linewidth=2, alpha=0.5)
axes[1].set_ylabel('Net Transfer Entropy', fontsize=12)
axes[1].set_title('Distribution Comparison: ΔTE vs ΔCTE', fontsize=14, fontweight='bold')
axes[1].set_xticks([1, 2])
axes[1].set_xticklabels(['ΔTE\n(unconditional)', 'ΔCTE\n(controlling for hour)'])
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

# Correlation
correlation = delta_comparison['Delta_TE'].corr(delta_comparison['Delta_CTE'])
print(f"\nCorrelation between ΔTE and ΔCTE: {correlation:.4f}")


## 5. Conclusion

### Summary of Findings

This analysis investigated the hypothesis that physical activity level (A) is more predictive of sitting behavior (S) than vice versa, using transfer entropy as a measure of directed information flow.

### Key Results:

1. **Primary Hypothesis (H₁: E[ΔTE] > 0)**:
   - *[Insert conclusion based on Wilcoxon test results above]*
   - Sample size: N subjects
   - Median ΔTE: [value]
   - p-value: [value]

2. **Robustness Check (Conditioning on Hour-of-Day)**:
   - *[Insert conclusion based on ΔCTE results above]*
   - Sample size: N subjects
   - Median ΔCTE: [value]
   - p-value: [value]

3. **Practical Implications**:
   - *[If H₁ supported]* The asymmetric relationship suggests that monitoring physical activity patterns could help predict upcoming sedentary periods, informing the design of just-in-time interventions to reduce sedentary behavior.
   - *[If H₁ not supported]* The lack of asymmetry suggests that activity and sitting have bidirectional predictive relationships, indicating that interventions should consider both activity patterns and sitting patterns.

### Methodological Notes:

- **Variable Definitions**: Activity (A) was z-scored and discretized within each subject to account for inter-individual differences in activity levels.
- **History Length Optimization**: Parameter k was optimized per subject using Active Information Storage (AIS), following information-theoretic best practices.
- **Statistical Significance**: Permutation testing with 1000 surrogates was used to assess significance of individual TE/CTE values.
- **Population-Level Inference**: Wilcoxon signed-rank test was used for non-parametric testing of the population-level effect.

### Limitations:

1. Time series length and data quality varied across subjects
2. Discretization may lose some information in continuous activity patterns
3. Analysis assumes stationarity within subjects
4. ExtraSensory dataset is observational, not experimental

### Future Directions:

- Investigate temporal dynamics of the asymmetry (does it vary by time of day or day of week?)
- Examine individual differences in ΔTE (who shows stronger asymmetry?)
- Explore multivariate extensions (conditioning on other variables like location, phone use)
- Validate findings in independent datasets

---

**Analysis completed**: [Date]

**Codebase**: ExtraSensory Transfer Entropy Analysis

**References**: Proposal_WeixuanKong.pdf, JIDT library documentation
